# Notebook 4: Introduzione a RAG - Embedding e Retrieval Base

Implementare i componenti base di RAG: embedding, semantic search e retrieval semplice

---


## 1. Setup e Import

Importiamo le librerie necessarie e configuriamo l'ambiente.


In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model="llama3.2:3b"
embedder = "all-minilm:l6-v2" # nomic-embed-text

# Inizializza LLM (chat model per generazione)
llm = ChatOllama(
    model=model,
    temperature=0.7
)

# Inizializza Embeddings (per rappresentazione vettoriale)
# Usa Ollama con modello embedding locale
embeddings = OllamaEmbeddings(
    model=embedder
)

print("✅ Setup completato!")
print(f"LLM: llama3.2:3b")
print(f"Embeddings: {embedder}")


## 2. Cos'è un Embedding?

Creiamo embedding di testi per vedere come funziona la rappresentazione vettoriale.


In [ ]:
# Testi di esempio
testi = [
    "Quando posso ritirare il documento d'identità?",
    "Quando posso ritirare la carta d'identità?",
    "Quali sono gli orari di apertura del comune?",
    "Quali sono gli orari del comune?",
    "Come si cucina la pasta?",
    "Ricetta per pasta carbonara"
]

# Crea embedding per ogni testo
print("=== Creazione Embedding ===\n")
embeddings_list = []

for i, testo in enumerate(testi):
    embedding = embeddings.embed_query(testo)
    embeddings_list.append(embedding)
    print(f"Testo {i+1}: {testo[:50]}...")
    print(f"Embedding: [{embedding[0]:.4f}, {embedding[1]:.4f}, ..., {embedding[-1]:.4f}]")
    print(f"Dimensione: {len(embedding)} dimensioni\n")


## 3. Similarità Semantica

Calcoliamo la similarità tra testi usando cosine similarity.


In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    """Calcola cosine similarity tra due vettori"""
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Confronta similarità tra testi
print("=== Similarità Semantica (Cosine Similarity) ===\n")
print("Simili (stesso argomento):")
for i in [0, 1]:  # Testi simili: ritiro documento
    for j in [0, 1]:
        if i < j:
            sim = cosine_similarity(embeddings_list[i], embeddings_list[j])
            print(f"  '{testi[i][:40]}...' vs '{testi[j][:40]}...'")
            print(f"  Similarità: {sim:.4f}\n")

print("Diversi (argomento diverso):")
for i in [0, 4]:  # Testo documento vs testo pasta
    for j in [0, 4]:
        if i != j:
            sim = cosine_similarity(embeddings_list[i], embeddings_list[j])
            print(f"  '{testi[i][:40]}...' vs '{testi[j][:40]}...'")
            print(f"  Similarità: {sim:.4f}\n")


## 4. Knowledge Base Semplice

Creiamo una knowledge base di esempio (documenti FAQ) e creiamo embedding per ogni documento.


In [ ]:
# Knowledge base di esempio: FAQ su pratiche comunali
knowledge_base = [
    {
        "id": 1,
        "titolo": "Ritiro Documenti",
        "contenuto": "Il documento d'identità può essere ritirato dal lunedì al venerdì dalle 9:00 alle 13:00 presso l'ufficio anagrafe. È necessario presentare la ricevuta del pagamento e un documento di riconoscimento valido."
    },
    {
        "id": 2,
        "titolo": "Orari Uffici",
        "contenuto": "Gli uffici comunali sono aperti dal lunedì al venerdì dalle 9:00 alle 13:00 e il martedì e giovedì anche il pomeriggio dalle 15:00 alle 17:30. Chiusi sabato, domenica e giorni festivi."
    },
    {
        "id": 3,
        "titolo": "Certificati Online",
        "contenuto": "È possibile richiedere certificati online tramite il portale SPID o CIE. I certificati vengono emessi entro 3 giorni lavorativi e inviati via email o PEC."
    },
    {
        "id": 4,
        "titolo": "Anagrafe - Cambio Residenza",
        "contenuto": "Per il cambio di residenza è necessario presentare domanda all'ufficio anagrafe entro 20 giorni dal trasferimento. Servono: documento d'identità, codice fiscale e contratto di affitto o proprietà."
    },
    {
        "id": 5,
        "titolo": "Tassa Rifiuti - ISEE",
        "contenuto": "Per richiedere l'esenzione o riduzione TARI in base all'ISEE, è necessario presentare il modello ISEE presso l'ufficio tributi entro il 31 gennaio di ogni anno."
    }
]

# Crea embedding per ogni documento nella knowledge base
print("=== Creazione Knowledge Base con Embedding ===\n")
kb_embeddings = []

for doc in knowledge_base:
    embedding = embeddings.embed_query(doc["contenuto"])
    kb_embeddings.append({
        "id": doc["id"],
        "titolo": doc["titolo"],
        "contenuto": doc["contenuto"],
        "embedding": embedding
    })
    print(f"✅ Documento {doc['id']}: {doc['titolo']}")
    print(f"   Embedding: {len(embedding)} dimensioni\n")

print(f"Knowledge base pronta con {len(kb_embeddings)} documenti!")


## 5. Semantic Search Base

Implementiamo semantic search per trovare i documenti più rilevanti per una query.


In [ ]:
def semantic_search(query, kb_embeddings_list, top_k=3):
    """
    Cerca i documenti più simili alla query usando semantic search
    
    Args:
        query: Testo della query
        kb_embeddings_list: Lista di documenti con embedding
        top_k: Numero di risultati da restituire
    
    Returns:
        Lista di documenti più simili con score
    """
    # Crea embedding della query
    query_embedding = embeddings.embed_query(query)
    
    # Calcola similarità con tutti i documenti
    similarities = []
    for doc in kb_embeddings_list:
        sim = cosine_similarity(query_embedding, doc["embedding"])
        similarities.append({
            "id": doc["id"],
            "titolo": doc["titolo"],
            "contenuto": doc["contenuto"],
            "similarity": sim
        })
    
    # Ordina per similarità (decrescente)
    similarities.sort(key=lambda x: x["similarity"], reverse=True)
    
    # Restituisci top-k
    return similarities[:top_k]

# Test semantic search
print("=== Semantic Search - Test ===\n")

query1 = "Quando posso ritirare la carta d'identità?"
print(f"Query 1: {query1}\n")
risultati1 = semantic_search(query1, kb_embeddings, top_k=2)

for i, risultato in enumerate(risultati1, 1):
    print(f"Risultato {i}:")
    print(f"  Titolo: {risultato['titolo']}")
    print(f"  Similarità: {risultato['similarity']:.4f}")
    print(f"  Contenuto: {risultato['contenuto'][:100]}...\n")

print("-" * 60 + "\n")

query2 = "Quali sono gli orari del comune?"
print(f"Query 2: {query2}\n")
risultati2 = semantic_search(query2, kb_embeddings, top_k=2)

for i, risultato in enumerate(risultati2, 1):
    print(f"Risultato {i}:")
    print(f"  Titolo: {risultato['titolo']}")
    print(f"  Similarità: {risultato['similarity']:.4f}")
    print(f"  Contenuto: {risultato['contenuto'][:150]}...\n")


## 6. RAG Semplice

Combiniamo retrieval e generation per creare un sistema RAG base.


In [ ]:
def rag_simple(query, kb_embeddings_list, llm, top_k=3):
    """
    Sistema RAG semplice: retrieval + generation
    
    Args:
        query: Domanda dell'utente
        kb_embeddings_list: Knowledge base con embedding
        llm: Modello LLM per generazione
        top_k: Numero di documenti da recuperare
    
    Returns:
        Risposta generata con fonti
    """
    # Step 1: Retrieval - trova documenti rilevanti
    documenti_rilevanti = semantic_search(query, kb_embeddings_list, top_k=top_k)
    
    # Step 2: Costruisci contesto dai documenti
    contesto = "\n\n".join([
        f"Documento {doc['id']} - {doc['titolo']}:\n{doc['contenuto']}"
        for doc in documenti_rilevanti
    ])
    
    # Step 3: Costruisci prompt con contesto
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Sei un assistente che risponde alle domande basandoti SOLO sul contesto fornito.
Se la risposta non è nel contesto, dì 'Non ho informazioni sufficienti nel contesto fornito.'
Cita sempre il numero del documento quando possibile."""),
        ("human", """Contesto:
{context}

Domanda: {query}

Risposta:""")
    ])
    
    # Step 4: Genera risposta
    chain = prompt | llm | StrOutputParser()
    risposta = chain.invoke({"context": contesto, "query": query})
    
    # Step 5: Restituisci risposta con fonti
    fonti = [{"id": doc["id"], "titolo": doc["titolo"]} for doc in documenti_rilevanti]
    
    return {
        "risposta": risposta,
        "fonti": fonti
    }

# Test RAG
print("=== RAG Semplice - Test ===\n")

query = "Quando posso ritirare il documento d'identità?"
print(f"Query: {query}\n")
risultato = rag_simple(query, kb_embeddings, llm, top_k=2)

print(f"Risposta: {risultato['risposta']}\n")
print(f"Fonti:")
for fonte in risultato['fonti']:
    print(f"  - Documento {fonte['id']}: {fonte['titolo']}")


## 7. Test RAG con Diverse Query

Testiamo il sistema RAG con diverse domande per vedere come funziona.


In [ ]:
# Test con diverse query
query_test = [
    "Quali sono gli orari di apertura degli uffici?",
    "Come posso richiedere un certificato online?",
    "Cosa serve per il cambio di residenza?",
    "Come funziona l'esenzione TARI con ISEE?"
]

print("=== Test RAG con Diverse Query ===\n")

for i, query in enumerate(query_test, 1):
    print(f"{'='*60}")
    print(f"Query {i}: {query}\n")
    
    risultato = rag_simple(query, kb_embeddings, llm, top_k=2)
    
    print(f"Risposta: {risultato['risposta']}\n")
    fonti_unite = ', '.join([f"Doc {f['id']}" for f in risultato['fonti']])
    print(f"Fonti: {fonti_unite}\n")


## 8. Note e Best Practices

### Cosa abbiamo imparato:
1. **Embedding**: rappresentazione vettoriale del testo che cattura significato semantico
2. **Similarità semantica**: testi simili hanno embedding simili (cosine similarity)
3. **Semantic Search**: ricerca dei documenti basata su significato
4. **RAG Base**: Retrieval + Generation = sistema che risponde usando knowledge base

### Limitazioni di questo approccio:
- **Embedding in memoria**: Non scala a migliaia di documenti
- **No vector database**: Ricerca lineare (lenta su grandi dataset)
- **No chunking**: Usiamo documenti interi (non ottimale per documenti lunghi)
- **No persistenza**: Embedding vengono ricalcolati ogni volta

### Best Practices:
- **Dimensioni embedding**: 384-768 dimensioni è un buon compromesso
- **Top-k retrieval**: 3-5 documenti sono solitamente sufficienti
- **Prompt engineering**: Istruzioni chiare sul contesto riducono hallucination
- **Test query**: Valutare con query reali, non solo esempi

---

**Congratulazioni! Hai completato il Notebook 4! 🎉**

Nel prossimo notebook vedremo come usare Chroma (vector database) e chunking avanzato per creare un sistema RAG completo e scalabile.
